# **RVC起動用のcolab notebook**<br>


In [1]:
# ドライブのマウント(学習用の音声はGoogle Deiveに用意します)
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

## GPUの確認

以下のコマンドでGPUの確認ができない場合には、Google Colaboratoryのメニューから「**ランタイム - ランタイムの変更 - ハードウェア アクセラレータ**」で「**GPU**」を選択後に、保存し、再度コードを実行してみてください。

In [2]:
# GPUの確認
!nvidia-smi
!nvcc -V
!free -h

Mon May 13 13:12:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## **必要なライブラリの準備**
- 現時点(2024年5月13日)時点で動作するものとなっています。今後のColabの更新で動かなくなる可能性もありますのでご注意下さい。





In [3]:
!pip3 install jedi==0.19.1
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install torch==2.1.0 torchtext==0.16.0 torchvision==0.16.0 torchaudio==2.1.0
!pip3 install optax==0.2.1
!pip3 install chex==0.1.7
!pip3 install pandas-stubs==2.0.1.230501
!pip3 install kaleido==0.2.1 fastapi==0.88 cohere==4.39 tiktoken==0.5.2 tensorflow-probability==0.20.1 typing-extensions==4.6.0 openai==1.5.0 python-multipart==0.0.6 uvicorn==0.25.0
!pip3 install faiss-cpu==1.7.3 fairseq gradio==3.34.0 ffmpy==0.3.1 ffmpeg-python>=0.2.0 praat-parselmouth>=0.4.2 pyworld==0.3.2 numpy==1.23.5 numba==0.56.4 librosa==0.9.1 tensorboardX tensorboard onnxruntime-gpu==1.16.3 onnx==1.15.0 torchcrepe==0.0.20 python-dotenv>=1.0.0 av
!pip3 install gradio_client==0.8.1
!pip3 install joblib>=1.1.0 scipy llvmlite==0.39.0 fairseq==0.12.2 Cython pydub>=0.25.1 soundfile>=0.12.1 Jinja2>=3.1.2 json5 Markdown matplotlib>=3.7.0 matplotlib-inline>=0.1.3 Pillow>=9.1.1 resampy>=0.4.2 scikit-learn tqdm>=4.63.1 tornado>=6.1 Werkzeug>=2.2.3 uc-micro-py>=1.0.1 sympy>=1.11.1 tabulate>=0.8.10 PyYAML>=6.0 pyasn1>=0.4.8 pyasn1-modules>=0.2.8 fsspec>=2022.11.0 absl-py>=1.2.0 audioread uvicorn>=0.21.1 colorama>=0.4.5 httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.

## **Retrieval-based-Voice-Conversion-WebUIのクローン**
GitHubから「Retrieval-based-Voice-Conversion-WebUI」のプログラムをGoogle Colaboratoryにクローンします。


In [4]:
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI

Cloning into 'Retrieval-based-Voice-Conversion-WebUI'...
remote: Enumerating objects: 5078, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (257/257), done.
remote: Total 5078 (delta 213), reused 289 (delta 137), pack-reused 4678
Receiving objects: 100% (5078/5078), 14.51 MiB | 26.57 MiB/s, done.
Resolving deltas: 100% (3120/3120), done.
/content/Retrieval-based-Voice-Conversion-WebUI


In [5]:
# gitの状態を最新にします
!git pull

Already up to date.


## **事前学習モデルなどの準備**
RVCのv1モデル(pretrained_v1)とv2モデル（pretrained_v2）をダウンロードします。  
また人間の声とその他を分離する音源分離のモデルもダウンロードします。

In [6]:
!apt -y install -qq aria2

#pretrained__v1の事前学習モデル
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0G48k.pth

#pretrained_v2の事前学習モデル
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0G48k.pth

#音源分離の学習モデル
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) 

In [7]:
# 「hubert_base.pt」・「rmvpe.pt」のダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/hubert/ -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/blob/main/rmvpe.pt -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/rmvpe/ -o rmvpe.pt


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
da704f|OK  |   240MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/assets/hubert//hubert_base.pt

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f500ee|OK  |   1.8MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/assets/rmvpe//rmvpe.pt

Status Legend:
(OK):download completed.


## **データセットと音声変換させたい音声ファイルの準備**<br>
[Google Drive](https://www.google.com/intl/ja_ALL/drive/) の「**MyDrive**」内に<br><br>
・「**dataset**」<br><br>
というボイスチェンジしたい音声ファイルが入っている学習予定のデータセットフォルダを用意します。<br>
**フォルダ構造**<br>
dataset<br>
　|---  〜1.wav<br>
　|---  〜2.wav<br>
　|---  〜3.wav<br>
　|---  〜4.wav<br>
　・<br>
　・<br>
　・<br>
　|--- 〜10.wav<br>
<br>

公式のQ＆Aより：
- 推奨される音源の長さは10〜50分で、高品質のトレーニングセット（特徴的な音　色、ノイズが少ない）ならば、5〜10分でも効果的
- 1〜2分のデータで成功した例もありますが、再現性が低いため推奨されない。
とあるので最低10分くらいは学習用のデータを用意した方が良さそうです。

学習回数は、<br>
・低品質でノイズの入った音声の場合には、20〜30エポック（学習回数）が推奨<br>
（低品質の音声で学習回数を増やしても音声の品質は向上しないとのこと）<br>
・高品質の音声では、学習回数の許容範囲としては200エポックとのこと<br><br>
→そのため、ただいたずらに音声データを集めるのではなく、高品質な音声データをいかに集めるのかが重要になってきます。

In [8]:
# 重複ファイル名の変換
# データセット（「**dataset**」フォルダ）内の重複ファイル名を変更
!ls -a /content/drive/MyDrive/dataset
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/drive/MyDrive/dataset/*.*~*

ls: cannot access '/content/drive/MyDrive/dataset': No such file or directory


## Web UIの起動

In [ ]:
# RVC WebUIの起動
# Running on public URL: https://〜.gradio.liveをクリックすることで起動することができる
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!python3 infer-web.py --colab --pycmd python3

/content/Retrieval-based-Voice-Conversion-WebUI
2024-05-13 13:23:49.664001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 13:23:49.664051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 13:23:49.791377: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 13:23:50.031324: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-

注：「**weights**」フォルダは「**assets**」フォルダ内にあります